In [1]:
import numpy as np
from csv import writer
import json

In [34]:
data = {}
with open('./results/2022-12-16 142018.578552.json') as result_file:
    data = json.load(result_file)

In [45]:
"""
data {
    'distance_matrix_time': distance_matrix_time,
    'employees': coordinates_list,
    'results': [
         'num_vehicles': n_shuttle,
         'max_time_shuttle': max_time,
         'result': {
            'routes': [{
                'route': [],
                'routeTotalTime': 0,
                'routeTotalDist': 0
                },
                ... more routes
            ]
            'objective': solution.ObjectiveValue()
            },
        ... more results
        ]
    ]
}
"""

{'routes': [{'route': [0, 15, 21, 4, 2, 1, 0],
   'routeTotalTime': 20422,
   'routeTotalDist': 242390},
  {'route': [0, 12, 19, 3, 8, 0],
   'routeTotalTime': 15610,
   'routeTotalDist': 227174},
  {'route': [0, 11, 18, 6, 9, 23, 20, 14, 0],
   'routeTotalTime': 22952,
   'routeTotalDist': 322785},
  {'route': [0, 13, 5, 24, 7, 10, 17, 16, 22, 0],
   'routeTotalTime': 24705,
   'routeTotalDist': 299741}],
 'objective': 561934}

In [2]:
dist_matrix = [[0, 1817, 1420, 1473, 1039, 882, 1608, 1431, 1507, 904],
                [1883, 0, 2113, 970, 1858, 1179, 537, 2046, 1010, 1737],
                [1457, 2022, 0, 1322, 457, 1399, 2039, 85, 1431, 2127],
                [1571, 813, 1350, 0, 1189, 858, 879, 1285, 199, 1432],
                [1074, 1867, 473, 1191, 0, 1004, 1649, 481, 1287, 1752],
                [1005, 1193, 1423, 856, 1042, 0, 1010, 1430, 896, 799],
                [1702, 494, 2056, 914, 1714, 982, 0, 1973, 952, 1468],
                [1459, 1952, 102, 1257, 454, 1388, 1973, 0, 1355, 2132],
                [1643, 881, 1414, 215, 1256, 933, 943, 1339, 0, 1511],
                [902, 1728, 2060, 1386, 1689, 761, 1382, 2069, 1422, 0]] #[m]

time_matrix = [[0, 1817, 1420, 1473, 1039, 882, 1608, 1431, 1507, 904],
                [1883, 0, 2113, 970, 1858, 1179, 537, 2046, 1010, 1737],
                [1457, 2022, 0, 1322, 457, 1399, 2039, 85, 1431, 2127],
                [1571, 813, 1350, 0, 1189, 858, 879, 1285, 199, 1432],
                [1074, 1867, 473, 1191, 0, 1004, 1649, 481, 1287, 1752],
                [1005, 1193, 1423, 856, 1042, 0, 1010, 1430, 896, 799],
                [1702, 494, 2056, 914, 1714, 982, 0, 1973, 952, 1468],
                [1459, 1952, 102, 1257, 454, 1388, 1973, 0, 1355, 2132],
                [1643, 881, 1414, 215, 1256, 933, 943, 1339, 0, 1511],
                [902, 1728, 2060, 1386, 1689, 761, 1382, 2069, 1422, 0]] #[s]

dist_matrix = np.array(dist_matrix)
time_matrix = np.array(time_matrix)


In [3]:
vrp_out = {'routes': [
    {'route': [0, 4, 5, 8, 0],
     'routeTotalTime': 5369,
     'routeTotalDist': 3985
    }, 
    {'route': [0, 1, 6, 2, 0],
     'routeTotalTime': 8133,
     'routeTotalDist': 1806
    }, 
    {'route': [0, 3, 7, 0],
     'routeTotalTime': 8042,
     'routeTotalDist': 3564
    }, 
    {'route': [0, 9, 0],
     'routeTotalTime': 5316,
     'routeTotalDist': 3033
    }], 
 'objective': 366036
}


In [23]:
### Dist & Emissions avoided 
# Inputs :  dist_matrix: the distance matrix
#           vrp_out: the results of the vrp
# Outputs:  - the avoided distance in meters
#           - the avoided emissions in gCO2eq

def fct_avoided_dist(dist_matrix, vrp_out):
    EMISSION_CAR     = 0.17 #[g/(m*passanger)]
    EMISSION_SHUTTLE = 0.10 #[g/(m*passanger)]

    FIRST_EMPLOYEE = 1
    COMPANY_COLUMN = 0

    avoided_dist = 0
    em_shuttles  = 0
    em_cars      = 0
    nb_employees = len(dist_matrix)

    for s in range(len(vrp_out['routes'])):
        
        dist_cars = 0
        for i in range(FIRST_EMPLOYEE, len(vrp_out['routes'][s]['route'])-1):
            dist_cars += dist_matrix[i][COMPANY_COLUMN]
        
        avoided_dist += (dist_cars - vrp_out['routes'][s]['routeTotalDist'])
        em_shuttles  += (vrp_out['routes'][s]['routeTotalDist'] * EMISSION_SHUTTLE)
        em_cars      += (dist_cars * EMISSION_CAR)

    avoided_em = (em_cars - em_shuttles)*nb_employees

    return(avoided_dist, avoided_em)



In [24]:
### Overall time of work loss
# Inputs :  time_matrix: the time matrix between all employees
#           vrp_time: the time result of the vrp
# Outputs:  - the overall time lost in seconds
#           - mean time lost
#           - the maximum time lost for all employees
#           - the minimum time lost for all employees

def fct_time_lost(time_matrix, vrp_out):
    FIRST_SHUTTLE  = 0
    FIRST_EMPLOYEE = 1
    COMPANY_COLUMN = 0
    
    tot_time_lost = 0
    max_time_lost = 0
    min_time_lost = vrp_out['routes'][FIRST_SHUTTLE]['routeTotalTime'] - time_matrix[vrp_out['routes'][FIRST_SHUTTLE]['route'][FIRST_EMPLOYEE]][COMPANY_COLUMN]
    nb_employees  = len(time_matrix)

    for s in range(len(vrp_out['routes'])):
        for i in range(FIRST_EMPLOYEE, len(vrp_out['routes'][s]['route'])-1):
            time_lost_i = (vrp_out['routes'][s]['routeTotalTime'] - time_matrix[i][COMPANY_COLUMN])
            tot_time_lost  += time_lost_i

            if time_lost_i > max_time_lost:
                max_time_lost = time_lost_i
            
            if time_lost_i < min_time_lost:
                min_time_lost = time_lost_i

    mean_time_lost = tot_time_lost/nb_employees

    return(tot_time_lost, mean_time_lost, max_time_lost, min_time_lost)

In [25]:
### Calculate & save resutls
# Inputs:   create_file: a boolean : true to create a new file (and erase previous one), False to add a row to the existing file
#           test_number: the name of the test, it will apprear on the first column of the output
#           dist_matrix: the distance matrix
#           time_matrix: the time matrix
#           vrp_out: the output dictionnary of the vrp
# Output:   - a print of the costs
#           - a file called "outputs-csv" with the costs
def fct_print_costs_save(create_file, test_number, dist_matrix, time_matrix, vrp_out):
    if create_file == True:
        with open('outputs.csv', 'w', newline='') as file:
            # 2. step
            w = writer(file)
            # 3. step
            w.writerow(["Test number; avoided dist [m]; avoided emissions [gCO2eq]; total time lost [s]; mean time lost [s]; max_time_lost [s]; min_time_lost[s]"])


    [avoided_dist, avoided_em] = fct_avoided_dist(dist_matrix, vrp_out)
    [tot_time_lost, mean_time_lost, max_time_lost, min_time_lost] = fct_time_lost(time_matrix, vrp_out)
    newrow = [str(test_number)+";" + str(avoided_dist)+";"+str(avoided_em)+";"+str(tot_time_lost)+";"+str(mean_time_lost)+";"+str(max_time_lost)+";"+str(min_time_lost)]

    with open(f'./results/metrics-outputs.csv', 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(newrow)

    print(
    "avoided distance -------- " + str(avoided_dist) + " [m]\n"
    "avoided emissions ------- " + str(avoided_dist) + " [gCO2eq]\n"
    "total time lost --------- " + str(tot_time_lost) + " [s]\n"
    "mean time lost ---------- " + str(mean_time_lost) + " [s]\n"
    "maximum time lost ------- " + str(max_time_lost) + " [s]\n"
    "minimum time lost ------- " + str(min_time_lost) + " [s]\n"
    )

In [26]:
fct_print_costs_save(True, 1, dist_matrix, time_matrix, vrp_out)

NameError: name 'dist_matrix' is not defined

In [35]:
fct_print_costs_save(True, 'Test 1 4 shuttles, 6000 max', data['distance_matrix_distance'], data['distance_matrix_time'], data['results'][0]['result'])

avoided distance -------- -620284 [m]
avoided emissions ------- -620284 [gCO2eq]
total time lost --------- 486903 [s]
mean time lost ---------- 19476.12 [s]
maximum time lost ------- 24118 [s]
minimum time lost ------- 13482 [s]



In [21]:
data['distance_matrix_distance'][i]

[[0,
  24311,
  24793,
  26775,
  35065,
  2537,
  20323,
  8706,
  24676,
  21275,
  17519,
  3313,
  23036,
  4923,
  16400,
  25536,
  13982,
  13026,
  19673,
  17406,
  15044,
  24701,
  9999,
  14241,
  3991],
 [24001,
  0,
  1512,
  30985,
  1902,
  28762,
  3932,
  31241,
  24000,
  4117,
  40054,
  27020,
  44323,
  28065,
  19886,
  8377,
  35269,
  34313,
  8487,
  25231,
  20844,
  2355,
  29359,
  12702,
  31621],
 [24073,
  507,
  0,
  31056,
  1973,
  28834,
  4004,
  31313,
  24071,
  4188,
  40126,
  27091,
  44394,
  28137,
  19957,
  8449,
  35340,
  34385,
  8558,
  25302,
  20916,
  2427,
  29431,
  12773,
  31692],
 [26645,
  31109,
  31591,
  0,
  29905,
  32062,
  27023,
  34542,
  8740,
  28073,
  43354,
  30320,
  12478,
  31365,
  10273,
  27985,
  17084,
  19409,
  24280,
  9409,
  11718,
  31499,
  16840,
  21846,
  34921],
 [20028,
  2156,
  2200,
  29762,
  0,
  38764,
  3726,
  41243,
  22777,
  4771,
  50056,
  37022,
  29534,
  38067,
  17848,
  9567,
